---
# PLC - ONS - 25/11/2025
---

# ETL - Python - OCR para Word e PDF (automate)

In [38]:
import pandas as pd
import io

# Define os prefixos que precisam ser removidos
# Esta lista é global para as funções, assim como no notebook original
prefixes_to_remove =  [
    '• Contingência Dupla das ',
    '• Contingência dupla das ',
    '• Contingência Dupla da ',
    'Contingência Dupla Das',
    'Contingência Dupla Da'
]

linhas_transmissao = ["LT", "KV"]

futura_prazo = {
    "Médio Prazo": "SIM",
    "Curto Prazo": "NÃO"
}



def clean_contingency_line(entry: str) -> str:
    """Aplica a lógica de limpeza a uma única linha de texto de contingência."""
    # Remove o marcador inicial (bullet point) e espaços em branco extras
    entry = entry.lstrip('• ').strip()

    # Remove os prefixos definidos
    for prefix in prefixes_to_remove:
        # Compara com o prefixo limpo para evitar problemas de espaços internos
        if entry.startswith(prefix.strip()):
            entry = entry[len(prefix.strip()):]
            break

    # Encontra a primeira ocorrência de 'LT' e extrai a string a partir daí
    lt_index = entry.find('LT')
    if lt_index != -1:
        entry = entry[lt_index:].strip()
    return entry

def process_contingency_data(data_source: str, is_file_path: bool = False) -> pd.DataFrame:
    """
    Processa dados de contingência de uma string multi-linhas ou de um arquivo .txt.

    Args:
        data_source (str): A string contendo as linhas de contingência
                           ou o caminho para um arquivo .txt.
        is_file_path (bool): Se True, data_source é tratado como um caminho de arquivo.
                             Caso contrário, é tratado como uma string direta.

    Returns:
        pd.DataFrame: Um DataFrame com a coluna original e a coluna processada.
    """
    lines = []
    if is_file_path:
        try:
            with open(data_source, 'r', encoding='utf-8') as f:
                lines = [line.strip() for line in f if line.strip()]
        except FileNotFoundError:
            print(f"Erro: Arquivo não encontrado em '{data_source}'")
            return pd.DataFrame(columns=['Linha Original', 'Linha Processada'])
    else:
        lines = [line.strip() for line in data_source.split('\n') if line.strip()]

    if not lines:
        return pd.DataFrame(columns=['Linha Original', 'Linha Processada'])

    df = pd.DataFrame(lines, columns=['Linha Original'])
    df['Linha Processada'] = df['Linha Original'].apply(clean_contingency_line)
    return df

def process_contingency_from_excel(excel_path: str, column_name: str) -> pd.DataFrame:
    """
    Processa dados de contingência de uma coluna específica em um arquivo Excel.

    Args:
        excel_path (str): O caminho para o arquivo Excel.
        column_name (str): O nome da coluna no Excel que contém os dados de contingência.

    Returns:
        pd.DataFrame: Um DataFrame com a coluna original e a coluna processada.
    """
    try:
        df_excel = pd.read_excel(excel_path)
    except FileNotFoundError:
        print(f"Erro: Arquivo Excel não encontrado em '{excel_path}'")
        return pd.DataFrame(columns=['Linha Original', 'Linha Processada'])
    except KeyError:
        print(f"Erro: Coluna '{column_name}' não encontrada no arquivo Excel.")
        return pd.DataFrame(columns=['Linha Original', 'Linha Processada'])

    if column_name not in df_excel.columns:
        print(f"Erro: Coluna '{column_name}' não encontrada no arquivo Excel.")
        return pd.DataFrame(columns=['Linha Original', 'Linha Processada'])

    df = pd.DataFrame(df_excel[column_name].astype(str).tolist(), columns=['Linha Original'])
    df['Linha Processada'] = df['Linha Original'].apply(clean_contingency_line)
    return df

print("Funções `process_contingency_data` e `process_contingency_from_excel` criadas com sucesso.")

########################################################################
def run_perdas_duplas_string():
    # Exemplo de uso da função process_contingency_data com uma string
    print("\n--- Exemplo: Processando string --- ")
    string_test = """
    • Contingência Dupla das LT 230 kV Teste A - Teste B
    • Contingência dupla das LT 138 kV Alpha - Beta
    • Contingência Dupla da LT 500 kV Gamma - Delta
    Uma linha sem prefixo ou LT
    LT 69 kV Zeta - Epsilon
    """
    df_string_processed = process_contingency_data(string_test)
    print(df_string_processed.to_string(index=False))
    df_string_processed[['Linha Processada']].to_excel(f'./df_string.xlsx', index=False)

def run_perdas_duplas_TXT():

    # Exemplo de uso da função process_contingency_data com um arquivo .txt (simulado)
    # Primeiro, vamos criar um arquivo .txt de exemplo para demonstração.
    example_txt_content = """
    • Contingência Dupla das LT 230 kV Arquivo TXT 1
    • Contingência dupla das LT 230 kV Arquivo TXT 2
    LT 230 kV Arquivo TXT 3
    """
    with open('exemplo_contingencias.txt', 'w', encoding='utf-8') as f:
        f.write(example_txt_content)

    print("\n--- Exemplo: Processando arquivo .txt --- ")
    df_txt_processed = process_contingency_data('exemplo_contingencias.txt', is_file_path=True)
    print(df_txt_processed.to_string(index=False))
    df_txt_processed[['Linha Processada']].to_excel(f'./df_txt.xlsx', index=False)





Funções `process_contingency_data` e `process_contingency_from_excel` criadas com sucesso.


## Tratamento de dados vindo de ums Docx e tratando apenas uma coluna de sublistas em regex, pandas e python

In [39]:
import pandas as pd
import io

# Simulando o carregamento dos dados da imagem em um DataFrame.
# Em um cenário real, você usaria pd.read_csv() ou pd.read_excel().
dados_texto = """Linha de Transmissao
Perda Dupla
Lt 525 Kv Abdon Batista - Siderópolis 2
Lt S Lt 525 Kv Blumenau - Gaspar 2 C2 e Lt 525 Kv Biguaçu - Gaspar 2
Lt S Lt 525 Kv Blumenau - Gaspar 2 C1 e Lt 525 Kv Curitiba - Gaspar 2
Lt 525 Kv Candiota 2 Guaíba 3 C1 e C2
Lt 525 Kv Capivari Do Sul - Gravataí e Capivari do Sul - Guaíba 3
Lt S Lt 525 Kv Guaíba 3 - Nova Santa Rita C1 e Lt 525 Kv Guaíba 3 - Povo Novo C1
Lt 525 Kv Guaíra - Sarandi
Lt 525 Kv Marmeleiro 2 - Povo Novo
Lt 525 Kv Sarandi - Londrina
Lt 525 Kv Caxias Norte - Itá e Caxias Norte - Caxias C2
Lt 525 Kv Curitiba Leste - Joinville Sul C1 E Lt 525 Kv Blumenau - Joinville Sul C1
Lt 525 Kv Curitiba - Joinville Sul C1 e Lt 525 Kv Joinville Sul - Gaspar 2 C2
Lt 525 Kv Blumenau - Gaspar 2 C1 e Lt 525 Kv Curitiba - Gaspar 2
Lt 525 Kv Blumenau - Gaspar 2 C2 e LT 525 Kv Biguaçu - Gaspar 2
Lt 525 Kv Joinville Sul - Gaspar 2 C2 e Lt 525 Kv Blumenau - Gaspar 2 C2
Lt S Lt 525 Kv Guaíba 3 Povo Novo C2 e C3
Lt S Lt 525 Kv Gravataí - Capivari Do Sul e Lt 525 Kv Guaíba 3 - Capivari do Sul
Lt S Lt 525 Kv Guaíba 3 - Gravataí e Lt 525 Kv Guaíba 3 - Capivari do Sul
Lt S Lt 525 Kv Campos Novos - Caxias Norte e Lt 525 Kv Caxias Norte - Caxias C1
Lt S Lt 525 Kv Itá - Caxias Norte e Lt 525 Kv Caxias Norte - Caxias C2."""

# Convertendo o texto simulado em um DataFrame
df = pd.read_csv(io.StringIO(dados_texto), header=None, names=['Linha Original'])

# --- Ação Principal: Limpeza da Coluna ---

# Definimos o nome da coluna que queremos limpar
nome_da_coluna = 'Linha Original'

# Usamos str.replace com regex para remover 'Lt S ' ou 'LT S ' do início
# O '^' indica o início da string
df['Linha Limpa'] = df[nome_da_coluna].str.replace(r'^(Lt S |LT S )', '', regex=True)
# Opcional: remover espaços extras que possam ter sobrado no início
df['Linha Limpa'] = df['Linha Limpa'].str.lstrip()

# Exibindo o resultado para comparação
print(df[['Linha Original', 'Linha Limpa']])

# Para salvar o resultado em um novo arquivo CSV:
# df[['Linha Limpa']].to_csv('linhas_limpas.csv', index=False)


                                       Linha Original  \
0                                Linha de Transmissao   
1                                         Perda Dupla   
2             Lt 525 Kv Abdon Batista - Siderópolis 2   
3   Lt S Lt 525 Kv Blumenau - Gaspar 2 C2 e Lt 525...   
4   Lt S Lt 525 Kv Blumenau - Gaspar 2 C1 e Lt 525...   
5               Lt 525 Kv Candiota 2 Guaíba 3 C1 e C2   
6   Lt 525 Kv Capivari Do Sul - Gravataí e Capivar...   
7   Lt S Lt 525 Kv Guaíba 3 - Nova Santa Rita C1 e...   
8                          Lt 525 Kv Guaíra - Sarandi   
9                  Lt 525 Kv Marmeleiro 2 - Povo Novo   
10                       Lt 525 Kv Sarandi - Londrina   
11  Lt 525 Kv Caxias Norte - Itá e Caxias Norte - ...   
12  Lt 525 Kv Curitiba Leste - Joinville Sul C1 E ...   
13  Lt 525 Kv Curitiba - Joinville Sul C1 e Lt 525...   
14  Lt 525 Kv Blumenau - Gaspar 2 C1 e Lt 525 Kv C...   
15  Lt 525 Kv Blumenau - Gaspar 2 C2 e LT 525 Kv B...   
16  Lt 525 Kv Joinville Sul - G

# Texto Volume copy

In [40]:
volume_7_string_table = """
    • Contingência Dupla das LT 230 kV Alegrete 2 - Livramento 2 e LT 230 kV Bagé 2 – Livramento 2 
    • Contingência Dupla das LT 230 kV Atlântida 2 - Gravataí 3 e LT 230 kV Atlântida 2 - Osório 2 
    • Contingência dupla LT 230 kV Cachoeirinha 3 – Cidade Industrial e LT 230 kV Cachoeirinha 3 – Canoas 2 
    • Contingência dupla LT 230 kV Cachoeirinha 3 – Cidade Industrial e LT 230 kV Canoas 2 – Cidade Industrial 
    • Contingência dupla LT 230 kV Cachoeirinha 3 – Gravataí 2 C1 e C2 
    • Contingência Dupla das LT 230 kV Camaquã 3 - N. Santa Rita e LT 230 kV N. Santa Rita - Porto Alegre 9 
    • Contingência Dupla das LT 230 kV Campo Bom – Caxias C1 e C2 
    • Contingência Dupla das LT 230 kV Campo Bom – Gravataí 2 C1 e C2 
    • Contingência Dupla das LT 230 kV N. Santa Rita - Candelária 2 e LT 230 kV Lajeado 2 - Nova Santa Rita 
    • Contingência Dupla das LT 230 kV N. Santa Rita - Candelária 2 e LT 230 kV Candelária 2 - UHE Itaúba 
    • Contingência Dupla das LT 230 kV Cidade Industrial - N. Santa Rita C1 e LT 230 kV Cidade Industrial - N. Santa Rita C2 
    • Contingência Dupla das LT 230 kV Cidade Industrial - N. Santa Rita C1 e LT 230 kV Cidade Industrial - Polo Petroquímico C1 
    • Contingência Dupla das LT 230 kV Cidade Industrial - N. Santa Rita C3 e LT 230 kV Cidade Industrial - Venâncio Aires 
    • Contingência Dupla das LT 230 kV Cidade Industrial - Polo Petroquímico e LT 230 kV Polo Petroquímico - N. Santa Rita 
    • Contingência Dupla das LT 230 kV Campo Bom – Gravataí 2 C1 e LT 230 kV Campo Bom – Gravataí 2 C2 
    • Contingência Dupla das LT 230 kV Caxias - Caxias 6 e LT 230 kV Caxias - Nova Petrópolis 2 
    • Contingência Dupla das LT 230 kV Caxias – Farroupilha C1 e LT 230 kV Caxias – Farroupilha C2 
    • Contingência Dupla das LT 230 kV Charqueadas 3 – Cidade Industrial e LT 230 kV Charqueadas – Scharlau 2 
    • Contingência Dupla das LT 230 kV Charqueadas 3 – Santa Cruz 1 e LT 230kV Santa Cruz 1 – UHE Itaúba 
    • Contingência Dupla das LT 230 kV Charqueadas – Scharlau 2 e LT 230 kV Farroupilha – Scharlau 2 
    • Contingência Dupla da LT 230 kV Charqueadas 3 – Cidade Industrial e LT 230 kV Cidade Industrial – Guaíba 2  
    • Contingência Dupla das LT 230 kV Canoas 1 - Porto Alegre 9 e LT 230 kV Cidade Industrial – Porto Alegre 9 
    • Contingência Dupla das LT 230 kV Cidade Industrial - Canoas 1 e LT 230 kV Cidade Industrial – Porto Alegre 9 
    • Contingência Dupla das LT 230 kV Cidade Industrial - Gravataí 2 C2 e LT 230 kV Canoas 2 - Cidade Industrial 
    • Contingência Dupla das LT 230 kV Cidade Industrial - Gravataí 2 C3 e LT 230 kV Cidade Industrial – Gravataí 2 C4 
    • Contingência Dupla das LT 230 kV Cidade Industrial - Guaíba 2 e LT 230 kV Charqueadas - Cidade Industrial 
    • Contingência Dupla das LT 230 kV UHE Dona Francisca - Santa Maria 3 C1 e LT 230 kV UHE Dona Francisca - UHE Itaúba C1 
    • Contingência Dupla das LT 230 kV Farroupilha – Scharlau 2 e LT 230 kV Farroupilha - Garibaldi 1 
    • Contingência dupla das LT 230 kV Guaíba 2 - Guaíba 3 C1 e C1 
    • Contingência Dupla das LT 230 kV Gravataí 2 - Gravataí 3 C1 e LT 230 kV Gravataí 2 - Gravataí 3 C2 
    • Contingência Dupla das LT 230 kV Gravataí 2 - Gravataí 3 C2 e LT 230 kV Gravataí 3 - Osório 2 (Ramal Fibraplac) 
    • Contingência Dupla das LT 230 kV Gravataí 2 – Porto Alegre 6 C1 e LT 230 kV Gravataí 2 – Porto Alegre 6 C2 
    • Contingência Dupla das LT 230 kV Gravataí 2 - Jardim Botânico e LT 230 kV Gravataí 2 - Viamão 3 
    • Contingência Dupla das LT 230 kV Gravataí 2 - Jardim Botânico e LT 230 kV Porto Alegre 6 – Viamão 3 
    • Contingência Dupla das LT 230 kV Gravataí 2 - Jardim Botânico e LT 230 kV Porto Alegre 4 - Porto Alegre 6 
    • Contingência Dupla das LT 230 kV Jardim Botânico - Porto Alegre 10 e LT 230 kV Porto Alegre 4 - Porto Alegre 6 
    • Contingência Dupla das LT 230 kV Lajeado 2 - N. Santa Rita e LT 230 kV Lajeado 2 - UHE Passo Real 
    • Contingência Dupla das LT 230 kV Livramento 3 – Santa Maria 3 C1 e C2 
    • Contingência Dupla das LT 230 kV Lajeado 2 - UHE Passo Real e LT 230 kV Venâncio Aires – UHE Passo Real 
    • Contingência Dupla das LT 230 kV N. Santa Rita - Scharlau 2 C1 e LT 230 kV N. Santa Rita - Scharlau 2 C2 
    • Contingência Dupla das LT 230 kV Porto Alegre 4 - Porto Alegre 6 e LT 230 kV Porto Alegre 4 - Porto Alegre 10 
    • Contingência Dupla das LT 230 kV Passo Fundo - Santa Marta e LT 230 kV Tapera 2 - Santa Marta 
    • Contingência Dupla das LT 230 kV Polo Petroquímico - UHE Itaúba e LT 230 kV N. Santa Rita - Polo Petroquímico 
    • Contingência Dupla das LT 230 kV Santa Rosa 1 - Santo Ângelo 2 e LT 230 kV Santo Ângelo - Santo Ângelo 2 
    • Contingência Dupla das LT 230 kV Tapera 2 - UHE Passo Real e LT 230 kV UHE Itaúba - UHE Passo Real 
    • Contingência Dupla das LT 230 kV Farroupilha - Caxias 5 e LT 230 kV Lajeado Grande – Caxias 5 
    • Contingência Dupla das LT 230 kV Garibaldi 1 – Vinhedos e LT 230 kV Monte Claro – Vinhedos 
    • Contingência Dupla das LT 230 kV Garibaldi 1 – Vinhedos e LT 230 kV Farroupilha – Garibaldi 1 
    • Contingência Dupla das LT 230 kV Porto Alegre 13 – Restinga e LT 230 kV Porto Alegre 6 - Porto Alegre 13 
    • Contingência Dupla das LT 230 kV Vila Maria – Nova Prata 2 C2 e LT 230 kV Passo Fundo - Vila Maria C2 
    • Contingência Dupla das LT 230 kV Vila Maria – Nova Prata 2 C1 e LT 230 kV Passo Fundo - Vila Maria C1
    • Contingência Dupla das LT 230 kV Maçambará 3 - Santo Ângelo C1 e LT 230 kV Maçambará 3 - Maçambará C1 
    • Contingência Dupla das LT 230 kV Maçambará 3 - Santo Ângelo C2 e LT 230 kV Maçambará 3 - Maçambará C1 
    • Contingência Dupla das LT 230 kV LT 230 kV Maçambará 3 - Maçambará C2 e LT 230 kV Maçambará 3 - Santo Ângelo C1
    • Contingência Dupla das LT 230 kV LT 230 kV Maçambará 3 - Maçambará C2 e LT 230 kV Maçambará 3 - Santo Ângelo C2 
    • Contingência Dupla das LT 230 kV Monte Claro – Nova Prata 2 C1 e LT 230kV Nova Prata 2 – Vila Maria 1 C1
    • Contingência Dupla das LT 230 Charqueadas 3 – Charqueadas e LT 230 Charqueadas 3 – Cidade Industrial  
    • Contingência Dupla das LT 230 kV Cachoeirinha 3 – Gravataí 2 C1 e C2 
    • Contingência Dupla das LT 230 kV Charqueadas 3 – Charqueadas e LT 230kV Charqueadas 3 – Cidade Industrial  
    • Contingência Dupla das LT 230 Charqueadas 3 – Santa Cruz 1 e LT 230 kV Charqueadas 3 – Charqueadas 
    • Contingência Dupla das LT 230 kV Caxias Norte – Farroupilha C2 e LT 230 kV Caxias Norte – Caxias do Sul 2 
    • Contingência Dupla das LT 230 kV Caxias Norte – Farroupilha C1 e LT 230 kV Caxias Norte – Caxias do Sul 5 
    • Contingência Dupla das LT 230 Charqueadas 3 – Charqueadas e LT 230 Charqueadas 3 – Cidade Industrial  
    • Contingência Dupla das LT 230 Charqueadas 3 – Santa Cruz 1 e LT 230 kV Charqueadas 3 – Charqueadas 
    • Contingência Dupla da LT 230 kV Caxias Norte – Caxias do Sul 5 e LT 230 kV Lajeado Grande – Caxias do Sul 5 
    • Contingência Dupla da LT 230 kV Caxias do Norte – Caxias do Sul 5 e LT 230 kV Caxias Norte – Farroupilha C1 
    • Contingência Dupla da LT 230 kV Caxias do Sul 2 – Caxias Norte e LT 230 kV Caxias do Sul 5 – Caxias Norte 
"""

In [41]:

NUM_VOLUME = 13


volume_7_string_table = """
•	Contingência Dupla das LT 345 kV Nova Ponte – Uberlândia 10 e Nova Ponte – Araxá 3 
•	Contingência Dupla das LT 345 kV Várzea da Palma 4 – Pirapora 2 e Várzea da Palma 4 – Várzea da Palma 1 
•	Contingência Dupla das LT 230 kV Itabira 2 – Itabira 5 C1 e Itabira 4 – Itabira 5 
•	Contingência Dupla das LT 230 kV Itabira 2 – Itabira 5 C2 e Itabira 4 – Itabira 5 
•	Contingência Dupla das LT 230 kV Janaúba 3 – Jaíba C1 e C2 
•	Contingência Dupla das LT 230 kV Baguari – Governador Valadares 2 e Baguari – Mesquita 
•	Contingência Dupla das LT 230 kV Braúnas – Porto Estrela e Mesquita - Porto Estrela 
•	Contingência Dupla da LT 500 kV Pirapora 2 – Buritizeiro 3 C1 e C2
•	Contingência Dupla da LT 500 kV Presidente Juscelino – Vespasiano 2 C1 e C2
•	Contingência Dupla da LT 500 kV Nova Ponte 3 – Araraquara 2 C1 e C2
•	Contingência Dupla da LT 500 kV Jaíba – Janaúba 6 C1 e C2
•	Contingência Dupla da LT 500 kV Janaúba 6 – Janaúba 3 C1 e C2
•	Contingência Dupla da LT 500 kV Janaúba 6 – Capelinha 3 C1 e C2
•	Contingência Dupla da LT 500 kV Capelinha 3 – Governador Valadares 6 C1 e C2
•	Contingência Dupla da LT 500 kV Jaíba – Buritizeiro 3 C1 e C2
•	Contingência Dupla da LT 500 kV Bom Jesus da Lapa II – Jaíba C1 e C2
•	Contingência Dupla da LT 500 kV Jussiape – São João do Paraíso C1 e C2


"""





# Tratamento de contigencias multiplas em perdas duplas

In [42]:
import pandas as pd

# Split the multiline string into individual lines and remove empty ones
lines = [line.strip() for line in volume_7_string_table.split('\n') if line.strip()]

# Create a DataFrame from the list of lines
df_volume_7 = pd.DataFrame(lines, columns=['Contingência Dupla'])

print("Initial DataFrame created successfully with the 'Contingência Dupla' column.")
print(df_volume_7.head())


prefixes = [
    '• Contingência Dupla das ',
    '• Contingência dupla das ',
    '• Contingência Dupla da ',
    'Contingência Dupla Das',
    'Contingência Dupla Da'
]


def substituir_string(text, query: list):
    # quero substituir LTs por LT

    return text.replace(query[0], query[1])

def clean_line_entry(entry):
    # Remove the initial bullet point and any leading/trailing whitespace
    entry = entry.lstrip('• ').strip()
    for prefix in prefixes:
        if entry.startswith(prefix.strip()):
            entry = entry[len(prefix.strip()):]
            break

    # Find the first occurrence of 'LT' and extract from there
    lt_index = entry.find('LT')
    if lt_index != -1:
        entry = entry[lt_index:].strip()

    # Substituir LTs por LT
    entry = substituir_string(entry, ['LTs', 'LT'])

    return entry





df_volume_7[f'Linha Processada Volume: {NUM_VOLUME}'] = df_volume_7['Contingência Dupla'].apply(clean_line_entry)

print("DataFrame with processed lines:")
#print(df_volume_7.head())




Initial DataFrame created successfully with the 'Contingência Dupla' column.
                                  Contingência Dupla
0  •\tContingência Dupla das LT 345 kV Nova Ponte...
1  •\tContingência Dupla das LT 345 kV Várzea da ...
2  •\tContingência Dupla das LT 230 kV Itabira 2 ...
3  •\tContingência Dupla das LT 230 kV Itabira 2 ...
4  •\tContingência Dupla das LT 230 kV Janaúba 3 ...
DataFrame with processed lines:


# Resultados

In [43]:


print(f"Displaying the 'Linha Processada' Volume: {NUM_VOLUME}")

#print(df_volume_7[['Linha Processada']].head())

# To display the full DataFrame column as a table, one might use to_string() or to_markdown()
print("\n\n\n")
display(df_volume_7[[f'Linha Processada Volume: {NUM_VOLUME}']])
# print(df_volume_7[['Linha Processada']].to_markdown(index=False))

df_volume_7[[f'Linha Processada Volume: {NUM_VOLUME}']].to_excel(f'./df_volume.xlsx', index=False)
print("Tabela gerada com suscesso")

Displaying the 'Linha Processada' Volume: 13






,Linha Processada Volume: 13
0,LT 345 kV Nova Ponte – Uberlândia 10 e Nova Po...
1,LT 345 kV Várzea da Palma 4 – Pirapora 2 e Vár...
2,LT 230 kV Itabira 2 – Itabira 5 C1 e Itabira 4...
3,LT 230 kV Itabira 2 – Itabira 5 C2 e Itabira 4...
4,LT 230 kV Janaúba 3 – Jaíba C1 e C2
5,LT 230 kV Baguari – Governador Valadares 2 e B...
6,LT 230 kV Braúnas – Porto Estrela e Mesquita -...
7,LT 500 kV Pirapora 2 – Buritizeiro 3 C1 e C2
8,LT 500 kV Presidente Juscelino – Vespasiano 2 ...
9,LT 500 kV Nova Ponte 3 – Araraquara 2 C1 e C2


Tabela gerada com suscesso


In [ ]:
def mapear_prazo_para_futura(df, coluna_prazo='Prazo', coluna_saida='Futura'):
    """
    Mapeia valores da coluna de prazo para SIM/NÃO na coluna Futura.
    
    Args:
        df: DataFrame a ser modificado
        coluna_prazo: Nome da coluna com os valores 'Curto Prazo' e 'Médio Prazo'
        coluna_saida: Nome da coluna onde será armazenado o resultado
    
    Returns:
        DataFrame modificado
    """
    df[coluna_saida] = df[coluna_prazo].map({
        'Médio Prazo': 'SIM',
        'Curto Prazo': 'NÃO'
    })
    
    return df


df_perdas_duplas = pd.read_excel(r"C:\Users\pedrovictor.veras\OneDrive - Operador Nacional do Sistema Eletrico\Documentos\ESTAGIO_ONS_PVRV_2025\GitHub\Palkia-PDF-extractor\src\BulbassaurQT6-ETL\sistema-ferramentas-RPA-desktop\app\assets\planilhas_PLC\perdas_duplas_ETL.xlsx" , sheet_name='Perdas Duplas Copy')


# Uso:
print("Corrigindo coluna Futura e Prazo...")
df = mapear_prazo_para_futura(df_perdas_duplas, coluna_prazo='Prazo', coluna_saida='Futura')
df.head()

df.to_excel(r"C:\Users\pedrovictor.veras\OneDrive - Operador Nacional do Sistema Eletrico\Documentos\ESTAGIO_ONS_PVRV_2025\GitHub\Palkia-PDF-extractor\src\BulbassaurQT6-ETL\sistema-ferramentas-RPA-desktop\app\assets\planilhas_PLC\perdas_duplas_ETL_corrigido.xlsx", index=False)

Corrigindo coluna Futura e Prazo...


,Volume,Área Geoelétrica,Perda Dupla,Prazo,Futura,Contingência Dupla,Página,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,Volume 2,Interligação Sul e Sudeste/Centro-Oeste,LT 765 kV Foz Do Iguaçu – Ivaiporã C1 e C2,Curto Prazo,NÃO,NÃO,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Volume 2,Interligação Sul e Sudeste/Centro-Oeste,LT 765 kV Foz Do Iguaçu – Ivaiporã C1 e C3,Curto Prazo,NÃO,NÃO,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Volume 2,Interligação Sul e Sudeste/Centro-Oeste,LT 765 kV Foz Do Iguaçu – Ivaiporã C2 e C3,Curto Prazo,NÃO,NÃO,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Volume 2,Interligação Sul e Sudeste/Centro-Oeste,LT 765 kV Ivaiporã – Itaberá C1 e C2,Curto Prazo,NÃO,NÃO,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Volume 2,Interligação Sul e Sudeste/Centro-Oeste,LT 765 kV Ivaiporã – Itaberá C2 e C3,Curto Prazo,NÃO,NÃO,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
